In [4]:
!pip install BeautifulSoup4

In [6]:
import requests
from bs4 import BeautifulSoup
import re

In [21]:
# Extract all links
course_links = []
for a_tag in soup.find_all("a", href=True):
    href = a_tag["href"]

    # Filter only course links (assuming course links start with "/courses/")
    if href.startswith("/courses/"):
        course_links.append(href)

paths = []
# Print extracted links
for link in course_links:
  paths.append(link)

In [24]:
pattern = re.compile(r"^/courses/[\w-]+/(?!.*\.pdf$)")
# Filter paths that match the regex
filtered_paths = [path for path in paths if pattern.match(path)]


['/courses/acct_i_s/', '/courses/act_sci/', '/courses/afroamer/', '/courses/african/', '/courses/a_a_e/', '/courses/agroecol/', '/courses/agronomy/', '/courses/a_f_aero/', '/courses/amer_ind/', '/courses/anatomy/', '/courses/anat_phy/', '/courses/anesthes/', '/courses/an_sci/', '/courses/anthro/', '/courses/abt/', '/courses/art/', '/courses/art_ed/', '/courses/art_hist/', '/courses/asian_am/', '/courses/asian/', '/courses/asialang/', '/courses/astron/', '/courses/atm_ocn/', '/courses/biochem/', '/courses/bse/', '/courses/biology/', '/courses/biocore/', '/courses/b_m_e/', '/courses/biomdsci/', '/courses/bmolchem/', '/courses/b_m_i/', '/courses/botany/', '/courses/crb/', '/courses/cbe/', '/courses/chem/', '/courses/chicla/', '/courses/civ_engr/', '/courses/cscs/', '/courses/classics/', '/courses/cnp/', '/courses/com_arts/', '/courses/cs_d/', '/courses/c_e_soc/', '/courses/comp_bio/', '/courses/comp_lit/', '/courses/comp_sci/', '/courses/cnsr_sci/', '/courses/coun_psy/', '/courses/curric/

In [77]:
courses = [div for div in soup.find_all(class_="courseblock")]

In [84]:
from bs4 import BeautifulSoup
import pandas as pd


prefix = "https://guide.wisc.edu"

for path in filtered_paths:
  temp = requests.get(prefix+path)
  # Parse HTML
  soup = BeautifulSoup(temp.content, "html.parser")

  # Extract course details

  for block in soup.find_all("div", class_="courseblock"):
      course_code = block.find("span", class_="courseblockcode").get_text(strip=True) if block.find("span", class_="courseblockcode") else "N/A"
      course_title = block.find("p", class_="courseblocktitle").get_text(strip=True) if block.find("p", class_="courseblocktitle") else "N/A"
      credits = block.find("p", class_="courseblockcredits").get_text(strip=True) if block.find("p", class_="courseblockcredits") else "N/A"
      description = block.find("p", class_="courseblockdesc").get_text(strip=True) if block.find("p", class_="courseblockdesc") else "N/A"

      # Extract requisites
      requisites = "N/A"
      for extra in block.find_all("p", class_="courseblockextra"):
          if "Requisites" in extra.get_text():
              requisites = extra.find("span", class_="cbextra-data").get_text(strip=True)

      # Extract last taught term
      last_taught = "N/A"
      for extra in block.find_all("p", class_="courseblockextra"):
          if "Last Taught" in extra.get_text():
              last_taught = extra.find("span", class_="cbextra-data").get_text(strip=True)

      # Extract learning outcomes
      learning_outcomes = []
      for extra in block.find_all("p", class_="courseblockextra"):
          if "Learning Outcomes" in extra.get_text():
              outcomes = extra.find("span", class_="cbextra-data")
              if outcomes:
                  learning_outcomes = [line.strip() for line in outcomes.get_text("\n").split("\n") if line.strip()]

      # Store data
      courses.append({
          "Course Code": course_code,
          "Course Title": course_title,
          "Credits": credits,
          "Description": description,
          "Requisites": requisites,
          "Last Taught": last_taught,
          "Learning Outcomes": "; ".join(learning_outcomes)  # Store as a semicolon-separated string
      }
      )
  print("Parses for ", path)

# Convert to DataFrame
df = pd.DataFrame(courses)

# Save to CSV
df.to_csv("courses.csv", index=False)

print("CSV file 'courses.csv' has been created successfully!")


CSV file 'courses.csv' has been created successfully!
